# Load and Save Files for CanRCM4 Downscaling Project
Load in and process HRDPS files here then export them as netcdf or numpy arrays

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path

In [2]:
#HRDPS data for all of 2016 to end of 2019
path = Path("/home/rbeutel/analysis/eosc510/proj.ipynb").resolve().parents[2]
path2 = "/results/forcing/atmospheric/GEM2.5/operational/"
path = path/path2

hrdps = xr.open_mfdataset(sorted(path.glob("ops_y201*.nc")))

/home/rbeutel/anaconda3/envs/eosc510/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/rbeutel/anaconda3/envs/eosc510/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[k

In [3]:
#next we want the HRDPS data to be daily instead of hourly
hrdps = hrdps.resample(time_counter='D').mean(dim='time_counter')

In [47]:
#find index of extra day (leap day)
ind = list(hrdps.time_counter.values).index(np.datetime64('2016-02-29T00:00:00.000000000'))

#and then trim to be in a more relevant extent and remove leap day
#(dont need the HRDPS data to stretch as far as it does inland)
hrdps = hrdps.sel(x=slice(0., 480000.),time_counter=slice('2015-12-31T00:00:00.000000000', '2019-12-31T00:00:00.000000000'))
hrdps_P = np.concatenate((hrdps.atmpres.values[0:ind,:,:], hrdps.atmpres.values[(ind+1):,:,:]))
hrdps_U = np.concatenate((hrdps.u_wind.values[0:ind,:,:], hrdps.u_wind.values[(ind+1):,:,:]))
hrdps_V = np.concatenate((hrdps.v_wind.values[0:ind,:,:], hrdps.v_wind.values[(ind+1):,:,:]))
hrdps_T = np.concatenate((hrdps.tair.values[0:ind,:,:], hrdps.tair.values[(ind+1):,:,:]))

In [48]:
#for some reason going crazy when i try to do hrdps.nav_lat.values[0] all in one step (saying array is of size (24744, 266, 256))
#dont know why but seems to help with doing in two steps
#ask doug about this
lat= hrdps.nav_lat[0]
lon= hrdps.nav_lon[0]

In [49]:
# set up titles
netcdf_title = 'HRDPSsubset.nc'
netcdf_comment = 'HRDPS2.5 datast used in CanRCM4 downscaling attempt for the Salish Sea'
notebook = 'LoadFiles.ipynb'

ds_attrs = {
        'creator_email':
            'rbeutel@eoas.ubc.ca',
        'institution_fullname': (
            'Earth, Ocean & Atmospheric Sciences,'
            ' University of British Columbia'
        ),
    'title': netcdf_title,
    'comment': netcdf_comment,
    'notebook': notebook,
    'summary': f'sea-level pressure, N/S wind, E/W wind, temperature',
    'history': (
            '[{}] File creation.'
            .format(dt.datetime.today().strftime('%Y-%m-%d'))
        )
}

coords_c = {
    'x' : np.arange(hrdps.x.shape[0]),
    'y' : np.arange(hrdps.y.shape[0]),
}

coords = {
    'x' : np.arange(hrdps.x.shape[0]),
    'y' : np.arange(hrdps.y.shape[0]),
    'time_counter' : np.concatenate((hrdps.time_counter.values[0:ind], hrdps.time_counter.values[(ind+1):]))
}

data_c = {}
var_attrs_c = {} 

data_c['nav_lat'] = lat.values
var_attrs_c['nav_lat'] = {'units': 'degrees_north',
                       'long_name': 'latitude'}

data_c['nav_lon'] = lon.values
var_attrs_c['nav_lon'] = {'units': 'degrees_east',
                       'long_name': 'longitude'}
#check shape of the navalt and navlon (may be 3 dim in which case add [0] to end)
da = {}
for var in data_c:
    da[var] = xr.DataArray(
        data = data_c[var],
        name=var,
        dims=('y', 'x'),
        coords = coords_c,
        attrs = var_attrs_c[var])

data = {}
var_attrs = {}

var_attrs['slp'] = {'units': 'Pa',
                      'long_name': 'Pressure Reduced to MLS [Pa]'}
data['slp'] = hrdps_P

var_attrs['u_wind'] = {'units': 'm/s',
                      'long_name': 'U component of wind [m/s]'}
data['u_wind'] = hrdps_U

var_attrs['v_wind'] = {'units': 'm/s',
                      'long_name': 'V component of wind [m/s]'}
data['v_wind'] = hrdps_V

var_attrs['temp'] = {'units': 'oC',
                      'long_name': 'Surface temperature [oC]'}
data['temp'] = hrdps_T

for var in data:
    da[var] = xr.DataArray(
        data = data[var],
        name=var,
        dims=('time_counter', 'y', 'x'),
        coords = coords,
        attrs = var_attrs[var])
    

ds = xr.Dataset(
        data_vars={
            'nav_lat': da['nav_lat'],
            'nav_lon': da['nav_lon'],
            'slp': da['slp'],
            'u_wind': da['u_wind'],
            'v_wind': da['v_wind'],
            'temp': da['temp']},
        coords = coords,
        attrs = ds_attrs
)

ds

<xarray.Dataset>
Dimensions:       (time_counter: 364, x: 193, y: 266)
Coordinates:
  * x             (x) int64 0 1 2 3 4 5 6 7 ... 185 186 187 188 189 190 191 192
  * y             (y) int64 0 1 2 3 4 5 6 7 ... 258 259 260 261 262 263 264 265
  * time_counter  (time_counter) datetime64[ns] 2016-01-02 ... 2016-12-31
Data variables:
    nav_lat       (y, x) float64 45.66 45.67 45.67 45.68 ... 51.97 51.97 51.98
    nav_lon       (y, x) float64 233.0 233.0 233.1 233.1 ... 237.2 237.3 237.3
    slp           (time_counter, y, x) float32 1.018e+05 1.018e+05 ... 1.02e+05
    u_wind        (time_counter, y, x) float32 -6.446 -6.539 ... 3.231 3.573
    v_wind        (time_counter, y, x) float32 6.22 6.242 ... -0.1155 0.04842
    temp          (time_counter, y, x) float32 283.0 283.0 282.9 ... 264.6 264.9
Attributes:
    creator_email:         rbeutel@eoas.ubc.ca
    institution_fullname:  Earth, Ocean & Atmospheric Sciences, University of...
    title:                 HRDPSsubset.nc
    comment:               HRDPS2.5 datast used in CanRCM4 downscaling attemp...
    notebook:              LoadFiles.ipynb
    summary:               sea-level pressure, N/S wind, E/W wind, temperature
    history:               [2020-12-30] File creation.

In [45]:
encoding = {var: {'zlib': True} for var in ds.data_vars}

ds.to_netcdf(netcdf_title, unlimited_dims=('time_counter'),
        encoding=encoding,)

<xarray.Dataset>
Dimensions:           (time_counter: 365, x: 193, y: 266)
Coordinates:
  * time_counter      (time_counter) datetime64[ns] 2016-01-02 ... 2016-12-31
  * x                 (x) float64 0.0 2.5e+03 5e+03 ... 4.775e+05 4.8e+05
  * y                 (y) float64 0.0 2.5e+03 5e+03 ... 6.6e+05 6.625e+05
Data variables:
    LHTFL_surface     (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    PRATE_surface     (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    RH_2maboveground  (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    atmpres           (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    nav_lat           (time_counter, y, x) float64 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    nav_lon           (time_counter, y, x) float64 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    percentcloud      (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    precip            (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    qair              (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    solar             (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    tair              (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    therm_rad         (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    u_wind            (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>
    v_wind            (time_counter, y, x) float32 dask.array<chunksize=(1, 266, 193), meta=np.ndarray>